In [1]:
import requests
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import time

In [2]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
data = requests.get(standings_url)

In [3]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [4]:
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])

In [5]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_60226/1811192786.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [6]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/possession/' in l]

In [7]:
data = requests.get(f"https://fbref.com{links[0]}")

In [8]:
possession = pd.read_html(data.text, match="Possession")[0]

/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_60226/2981814824.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  possession = pd.read_html(data.text, match="Possession")[0]


In [9]:
possession.columns = possession.columns.droplevel()

In [10]:
team_data1 = matches.merge(possession[["Date", "Touches", "Att 3rd", "Att Pen", "TotDist", "PrgDist", "PrgC", "CPA", "Rec", "PrgR"]], on="Date")

In [10]:
years = list(range(2024, 2020, -1))
all_matches = []
years

[2024, 2023, 2022, 2021]

In [11]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [12]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/possession/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        possession = pd.read_html(data.text, match="Possession")[0]
        possession.columns = possession.columns.droplevel()
        try:
            team_data = matches.merge(possession[["Date", "Touches", "Att 3rd", "Att Pen", "TotDist", "PrgDist", "PrgC", "CPA", "Rec", "PrgR"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(20)

/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_70003/2275031886.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_70003/2275031886.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  possession = pd.read_html(data.text, match="Possession")[0]
/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_70003/2275031886.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/i

In [9]:
len(all_matches)

100

In [13]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [14]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,att 3rd,att pen,totdist,prgdist,prgc,cpa,rec,prgr,season,team
0,2023-08-12,21:30,La Liga,Matchweek 1,Sat,Away,W,2,0,Athletic Club,...,154.0,24.0,2588.0,1342.0,21.0,4.0,551.0,48.0,2024,Real Madrid
1,2023-08-19,19:30,La Liga,Matchweek 2,Sat,Away,W,3,1,Almería,...,252.0,43.0,2771.0,1638.0,32.0,13.0,546.0,55.0,2024,Real Madrid
2,2023-08-25,21:30,La Liga,Matchweek 3,Fri,Away,W,1,0,Celta Vigo,...,178.0,24.0,2958.0,1519.0,28.0,8.0,583.0,42.0,2024,Real Madrid
3,2023-09-02,16:15,La Liga,Matchweek 4,Sat,Home,W,2,1,Getafe,...,295.0,52.0,3563.0,1861.0,31.0,9.0,668.0,84.0,2024,Real Madrid
4,2023-09-17,21:00,La Liga,Matchweek 5,Sun,Home,W,2,1,Real Sociedad,...,168.0,31.0,2558.0,1180.0,27.0,8.0,475.0,46.0,2024,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2021-05-01,14:00,La Liga,Matchweek 34,Sat,Home,W,3,0,Alavés,...,172.0,17.0,1037.0,429.0,9.0,2.0,217.0,29.0,2021,Eibar
37,2021-05-09,14:00,La Liga,Matchweek 35,Sun,Away,W,1,0,Getafe,...,141.0,15.0,1139.0,431.0,8.0,0.0,296.0,31.0,2021,Eibar
38,2021-05-13,20:00,La Liga,Matchweek 36,Thu,Home,D,1,1,Betis,...,256.0,25.0,2358.0,1076.0,16.0,1.0,485.0,59.0,2021,Eibar
39,2021-05-16,18:30,La Liga,Matchweek 37,Sun,Away,L,1,4,Valencia,...,265.0,36.0,2260.0,1096.0,28.0,4.0,489.0,58.0,2021,Eibar


In [15]:
match_df.to_csv("matches_possession.csv")